In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys,os
import tensorflow

from tensorflow.keras.layers import Input,Dense,LeakyReLU,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam

In [9]:
mnist = tensorflow.keras.datasets.mnist

(X_train,y_train),(X_test,y_test) =mnist.load_data()

# Map inputs
X_train,X_test = X_train/255*2-1,X_test/255*2-1
print(X_train.shape)

(60000, 28, 28)


In [10]:
# Flatten the data

N,H,W = X_train.shape
D = H*W
X_train = X_train.reshape(-1,D)
X_test = X_test.reshape(-1,D)

In [11]:
# Dimensionality of latent space

latent_dim = 100

In [12]:
# Get the generator model

def build_generator(latent_dim):
    i = Input(shape=(latent_dim,))
    x = Dense(256,activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(512,activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(1024,activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(D,activation='tanh')(x)
    
    model = Model(i,x)
    return model

In [13]:
def build_discriminator(latent_dim):
    i = Input(shape=(latent_dim,))
    x = Dense(256,activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(512,activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(1024,activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(D,activation='tanh')(x)
    
    model = Model(i,x)
    return model

In [14]:
# Compile both models in prep for training

# Building and compiling the discriminator

discriminator=build_discriminator(D)
discriminator.compile(loss='binary_crossentropy',optimizer=Adam(0.0002,0.5),metrics=['accuracy'])

# Build and compile model
generator = build_generator(latent_dim)
